<a href="https://colab.research.google.com/github/kkosmi/ybigta/blob/master/DL%EA%B8%B0%EC%B4%88_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 과제1: tensorflow를 이용한 NN 밑바닥부터 구현
input feature가 100개이고,  
hidden layer가 2개이고 neuron이 각각 50,10개이고,  
output이 5개인 NN를 구현해 보자  
* hidden layer는 relu를 activation function으로, output layer는 softmax를 activation function으로 사용

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
n_x = 100
n_h1 = 50
n_h2 = 10
n_y = 5

In [4]:
# Simulate train set
m = 500

x_train=np.random.randn(m,n_x).astype(np.float32)
y_train=np.zeros((m,n_y)).astype(np.float32)
y_train[np.arange(m),np.random.randint(n_y,size=m)]=1

print(x_train)
print(y_train)

[[-2.4148176  -0.12811758  2.1195862  ... -0.79631376 -0.80298257
  -0.23154707]
 [ 0.2650589  -0.5807347  -0.37117913 ... -0.7096159   0.08810216
  -0.5848298 ]
 [-0.66165787 -0.5598481   0.9231691  ...  1.4476304  -0.96571434
   0.38199127]
 ...
 [-0.64469165 -0.39725477 -0.4246492  ...  0.54368275 -1.0609186
   0.9549599 ]
 [ 0.48217833 -0.9190567   0.5009376  ...  1.4610416  -0.33776233
  -0.5260377 ]
 [ 0.99762315 -0.72476274 -0.57272995 ...  0.7682012  -2.3565116
  -0.5821261 ]]
[[0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


* Initialization of weights

In [5]:
w1=tf.Variable(1e-3*np.random.randn(n_x,n_h1).astype(np.float32),name="w1")
## 코드를 작성해 보세요 ## 
w2=tf.Variable(1e-3*np.random.randn(n_h1,n_h2).astype(np.float32),name="w2")
w3=tf.Variable(1e-3*np.random.randn(n_h2,n_y).astype(np.float32),name="w3")

* forward propagation을 통해 prediction 값을 구하고, loss를 구하는 function을 만들어 봅시다

In [6]:
def forward(x):
    z1=tf.matmul(x,w1)
    a1=tf.nn.relu(z1)
    ## 코드를 작성해 보세요 ##
    z2=tf.matmul(a1,w2)
    a2=tf.nn.relu(z2)
    z3=tf.matmul(a2,w3)
    predictions = tf.nn.softmax(z3)
    
    return predictions

def loss_fn(predictions, y):
    loss= -tf.reduce_sum(y*tf.math.log(predictions))
    return loss    

* backpropagation & update parameters

In [7]:
learning_rate=1e-2
optimizer=tf.keras.optimizers.Adam(learning_rate)

def train_step(x, y):
    with tf.GradientTape() as tape:
        predictions = forward(x)
        loss = loss_fn(predictions, y)
    ## 코드를 작성해 보세요 ## (hint: tape.gradient를 구글링 해보세요)
    gradient = tape.gradient(loss, [w1,w2,w3])
    # optimizer와 위에서 구한 경사도를 이용해 가중치들을 업데이트 합니다.
    optimizer.apply_gradients(zip(gradient, [w1, w2, w3]))
    return loss, w1, w2, w3


* 간단하게 train loop를 작성해 loss가 줄어나가는지 확인해 봅시다.

In [8]:
loss_list = []

for step in range(10): 
    loss, w1, w2, w3 = train_step(x_train, y_train)
    loss_list.append(loss.numpy())
    
print(loss_list)

[804.7187, 804.70416, 804.46796, 803.6229, 801.5674, 797.6154, 791.1305, 781.83636, 770.31903, 758.35394]


# 과제2: MNIST 데이터를 나만의 NN model로 95 % 이상의 성능으로 training 시켜보자!


## Loading MNIST training data

In [9]:
from tensorflow import keras

# import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Loading the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scaling(image data는 min-max scaling 주로 사용)
x_train = x_train/255.0
x_test = x_test/255.0

11493376/11490434 [==============================] - 0s 0us/step


## Training Data
28 * 28 pixel 값을 가진 총 60000개의 이미지 데이터

In [10]:
x_train.shape 

(60000, 28, 28)

Neural network 모델에 맞게 이미지 데이터를 벡터 형태로 데이터를 reshape 합니다.  
(Model을 만들 때 *keras.layers.Flatten(input_shape=(28, 28)) 이용해도 됨)

In [ ]:
x_train, x_test = x_train.reshape((-1, 28*28)), x_test.reshape((-1, 28*28))
x_train.shape, x_test.shape

In [ ]:
plt.imshow(x_train[0]).set_cmap('Greys')

## Training Labels
이미지 데이터가 나타내는 숫자값을 label로 가지고 있고, 0부터 9까지의 값을 나타냄  
마찬가지로, 60000개의 label이 존재

In [13]:
y_train.shape

(60000,)

In [14]:
# show MNIST label for above data
y_train[4]

9

## 나만의 모델을 tensorflow keras API 를 이용해 만들어 봅시다~

* parameters for model

In [ ]:
activation_list = ["sigmoid", "relu", "softmax", "tanh"]

loss_list = ["sparse_categorical_crossentropy",
             "categorical_crossentropy", 
             "binary_crossentropy"]

optimizer_list = ["sgd", "adam", "rmsprop", "adagrad"]

initializer_list = [tf.keras.initializers.RandomNormal(), 
                    tf.keras.initializers.RandomUniform(), 
                    tf.keras.initializers.he_normal(), 
                    tf.keras.initializers.he_uniform(), 
                    tf.keras.initializers.GlorotUniform(),
                    tf.keras.initializers.GlorotNormal()]

# dropout
dropout_rate = 0.3
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=784, activation = "sigmoid"),
    tf.keras.layers.Dense(2, activation = "sigmoid"),
    tf.keras.layers.Dropout(dropout_rate)
])


# regularizer
regularizer = tf.keras.regularizers.l1(1e-3)
regularizer = tf.keras.regularizers.l2(1e-3)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=784, activation="sigmoid",
                          activity_regularizer=regularizer)
])

# weight initialization
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=784, activation="sigmoid",
                          kernel_initializer=initializer_list[0])
])

#### My Own Model 

In [18]:
#### 자유롭게 Model을 만들고 compile 해봅시다 ####
model = keras.Sequential([
            keras.layers.Dense(784, kernel_initializer='he_normal', activity_regularizer=tf.keras.regularizers.l2(1e-3)),
            keras.layers.Dense(128, activation = 'relu', kernel_initializer='he_normal', activity_regularizer=tf.keras.regularizers.l2(1e-3)),
            keras.layers.Dense(10, activation = 'softmax')
])

In [20]:
model.compile(loss = "sparse_categorical_crossentropy", 
              optimizer = "adam", 
              metrics = ['accuracy'])

내가 만든 모델을 확인해 봅시다.

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (32, 784)                 615440    
_________________________________________________________________
dense_1 (Dense)              (32, 128)                 100480    
_________________________________________________________________
dense_2 (Dense)              (32, 10)                  1290      
Total params: 717,210
Trainable params: 717,210
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train, y_train,  epochs = 100)

Epoch 1/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3157 - accuracy: 0.9564
Epoch 2/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2880 - accuracy: 0.9615
Epoch 3/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2673 - accuracy: 0.9640
Epoch 4/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2631 - accuracy: 0.9648
Epoch 5/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2410 - accuracy: 0.9688
Epoch 6/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2374 - accuracy: 0.9686
Epoch 7/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2334 - accuracy: 0.9699
Epoch 8/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2185 - accuracy: 0.9720
Epoch 9/100
1875/1875 [==============================] - 10s 5ms/step - loss: 0.2239 - accuracy: 0.9706
Epoch 10/100
1875/1875 [==============================] - 10s 5m

model을 자유롭게 train 해봅시다.  

95%이상의 성능을 가진 모델을 만들면 완성!

In [26]:
test_loss, test_acc = model.evaluate(x_test,y_test, verbose=2)

print('\nAccuracy:', test_acc)

313/313 - 1s - loss: 0.3216 - accuracy: 0.9631

Accuracy: 0.963100016117096


![](https://www.tensorflow.org/versions/master/images/mnist_tensorboard.png)